In [93]:
#Alyssa Cox
#Machine Learning HW 2: Output


import pandas as pd
from patsy import dmatrices
import math
import numpy as np
from sklearn.linear_model import LogisticRegression

In [94]:
# For part 1: Read Data
def get_df(file):
    '''
    Reads a csv file into a pandas dataframe

    Inputs:
        file: CSV file to read, as a string

    Returns: A pandas data frame
    '''
    df = pd.read_csv(file)
    return df

In [95]:
#Let's read the credit data csv
df = get_df("credit-data.csv")
df

,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,60644,2,0.802982,9120.0,13,0,6,0,2.0
1,2,0,0.957151,40,60637,0,0.121876,2600.0,4,0,0,0,1.0
2,3,0,0.658180,38,60601,1,0.085113,3042.0,2,1,0,0,0.0
3,4,0,0.233810,30,60601,0,0.036050,3300.0,5,0,0,0,0.0
4,5,0,0.907239,49,60625,1,0.024926,63588.0,7,0,1,0,0.0
5,6,0,0.213179,74,60629,0,0.375607,3500.0,3,0,1,0,1.0
6,7,0,0.305682,57,60637,0,5710.000000,NaN,8,0,3,0,0.0
7,8,0,0.754464,39,60625,0,0.209940,3500.0,8,0,0,0,0.0
8,9,0,0.116951,27,60804,0,46.000000,NaN,2,0,0,0,NaN
9,10,0,0.189169,57,60629,0,0.606291,23684.0,9,0,4,0,2.0


In [96]:
# Some functions for part 2: Explore Data
def get_count_by_characteristic(file, characteristic):
    '''
    Gets counts, in a dataframe grouped by a
    certain characteristic.

    Inputs:
            file: the CSV file to be read, as a string
            characteristic: the characteristic you want to see counts by

    Returns: dataframe where the first column is the characteristic and
    the second column is the count.
    '''
    df_main = get_df(file)

    sorted = df_main.groupby(characteristic).groups
    list_0 = []
    list_1 = []
    for i in sorted:
        list_0.append(i)
        list_1.append(len(sorted[i]))

    new_df = pd.Series(list_1, list_0)
    return new_df

In [97]:
#We can see counts for each particular column broken down into categories--we see the distribution of values. 
#For example, what is the distribution of serious delinquencies in two years?
get_count_by_characteristic("credit-data.csv", "SeriousDlqin2yrs")

0    139974
1     10026
dtype: int64

In [98]:
#What about number of dependents?
get_count_by_characteristic("credit-data.csv", "NumberOfDependents")

0.0     86902
1.0     26316
2.0     19522
3.0      9483
4.0      2862
5.0       746
6.0       158
7.0        51
8.0        24
9.0         5
10.0        5
13.0        1
20.0        1
dtype: int64

In [99]:
#Most people have between 0 and 3 dependents. 13 and 20 dependents are obvious outliers. 

In [100]:
def get_counts_by_value(file, column, value):
    '''
    Returns the count for any value specified
    It's like get_count_by_characteristic,
    but returns the count for one value, not all of them.

    Inputs:
            file: CSV to be read, as a string
            column: the column containing the specific value
            value: the specific value you want a count for

    Returns: Count, an int
    '''
    df = get_df(file)
    index = df.columns.get_loc(column) + 1
    count = 0
    for tup in df.itertuples():
        if tup[index] == value:
            count += 1
    return count

In [101]:
#We can also see the counts for a particular value in a column. 
#How many people in the dataset are of age 45?
get_counts_by_value("credit-data.csv", "age", 45)

3502

In [102]:
#How many people are in the 60637 (Hyde Park) zipcode?
get_counts_by_value("credit-data.csv", "zipcode", 60637)

16625

In [103]:
def get_most(file, column):
    '''
    Gets the specific value from a specific column that has the
    most requests of all the unique values in that column.

    Inputs:
            file: CSV file to be read, as a string
            column: the column from which you want the
            value with the highest count.
    Returns: Tuple containing the count and the value.
                ex: (5600, 60615)
    '''

    df = get_df(file)
    grouped = df.groupby(column).groups
    list_most = []
    for group in grouped:
        list_most.append((len(grouped[group]), group))
    most = 0
    most_tup = None
    for item in list_most:
        if item[0] > most:
            most = item[0]
            most_tup = (item)
    return most_tup

def get_mean(df, column):
    '''
    Calculates the mean value for a given column.

    Inputs:
            df: a pandas dataframe
            column: specific column from which to generate a mean value

    Returns: mean, an integer
    '''

    if type(column) is str:
        index = df.columns.get_loc(column) + 1
    elif type(column) is int:
        index = column
    list_mean = []
    for tup in df.itertuples():
        if math.isnan(tup[index]):
            continue
        else:
            list_mean.append(tup[index])
    list_mean.sort()
    total = 0
    for i in list_mean:
        total = total + i
    mean = total / len(list_mean)
    new_mean = float("{:.2f}".format(mean))
    return new_mean

def get_max_and_min(df, column):
    '''
    Retrieves the max and min values from a specified column.

     Inputs:
            df: a pandas dataframe
            column: specific column from which to generate max and min

    Returns: a tuple containing (min, max)
    '''

    index = df.columns.get_loc(column) + 1
    list_vals = []
    for tup in df.itertuples():
        list_vals.append(tup[index])
    list_vals.sort()
    min = list_vals[0]
    max = list_vals[-1]
    return(min, max)

def get_stat_summ(file, column):
    '''
    Calculates the mean, median, and mode of a specific column

    Inputs:
            file: csv to be read
            column: column must have integer data

    Returns: string detailing the values of the mean, median, and mode
    '''
    df = get_df(file)

    mean = get_mean(df, column)

    index = df.columns.get_loc(column) + 1
    list_vals = []
    for tup in df.itertuples():
        list_vals.append(tup[index])
    list_vals.sort()

    if len(list_vals) % 2 == 0:
        length = len(list_vals)
        med_1 = list_vals[int(length / 2) - 1]
        med_2 = list_vals[int(length / 2)]
        median = (med_1 + med_2) / 2
    elif len(list_vals) % 2 == 1:
        length = len(list_vals)
        median = (length // 2)

    most_tup = get_most(file, column)
    mode = most_tup[1]
    return ("Mean for " + column + " is " + str(mean) + ", median is " + str(median) +
            " , mode is " + str(mode))



In [104]:
#Seeing a statistical summary of each column, including mean, median, mode, min, and max is helpful. 
#We can do this for every column, but it makes sense to use it on columns with continuous or categorical values. 
print(get_stat_summ("credit-data.csv", "DebtRatio"))
print(get_stat_summ("credit-data.csv", "age"))
print(get_stat_summ("credit-data.csv", "MonthlyIncome"))
print(get_stat_summ("credit-data.csv", "NumberOfDependents"))

Mean for DebtRatio is 353.01, median is 0.366507841 , mode is 0.0
Mean for age is 52.3, median is 52.0 , mode is 49
Mean for MonthlyIncome is 6670.22, median is nan , mode is 5000.0
Mean for NumberOfDependents is 0.76, median is 0.0 , mode is 0.0


In [105]:
#The high mean, along with the low median, for the DebtRatio points out a potential skew or the possibility of mistakes.


In [106]:
#Part 3: Preprocess and clean

def find_nans(df):
    col_nans = []
    headers = df.dtypes.index
    for row in df.itertuples():
            for i in range(len(row)):
                if math.isnan((row[i])):
                    col_name = headers[i - 1]
                    if col_name in col_nans:
                        continue
                    else:
                        col_nans.append(col_name)
    return col_nans

def fill_values(df, column, fill_val="mean"):
    '''
    Fills in NaN cell values with either the mean from that column or a pre-determined value

    Inputs:
            df: a pandas data frame
            column: column to fill in nan values
            fill_val: default is mean value of column, but can be specified by user for
            categorical or binary vars

    Returns: pandas data frame with no missing values for the column specified
    '''

    if fill_val == "mean":
        mean = get_mean(df, column)
        df[column].fillna(mean, inplace=True)
    else:
        df[column].fillna(fill_val, inplace=True)

    return df

# Part 4: Build Classifiers
def discretize(df, column, num_buckets):
    '''
    Turns a continuous variable into a discrete one by separating
    continuous values into intervals

    Inputs:
            df: a pandas data frame
            column: specific column to discretize (must have numerical data)
            num_buckets: the number of intervals to separate the data into

    Returns: pandas data frame where the value in the specified column is the maximum
    value in the interval into which the original data fell.
        ex: values range from 0 - 100, user specifies 10 buckets/intervals, [0-10],[11-20], etc.
            a row with the value 15 will fit into interval [11-20] and will thus be replaced
            with the number 20.
    '''
    index = df.columns.get_loc(column)
    min = get_max_and_min(df, column)[0]
    max = get_max_and_min(df, column)[1]
    partition = round((max - min) / num_buckets)
    for i in range(num_buckets):
        for row in df.itertuples():
            if row[index] > partition * (i) and row[index] < partition * (i + 1):
                range_part = partition * (i + 1)
                df.set_value(row[0], column, range_part)
    return df

def make_dummies(df, column_list):
    '''
       Turns a categorical variable into a binary/dummy variable.

       Inputs:
               df: a pandas dataframe
               columns: the columns to make binary, as a list

       Returns: dataframe with columns added that correspond to the dummy vars and values
       '''

    dummies_df = pd.get_dummies(df, columns=column_list)
    return dummies_df

In [64]:
# Again, let's use the credit data file. 
df = get_df("credit-data.csv")
find_nans(df)

['MonthlyIncome', 'NumberOfDependents']

In [65]:
# We have two columns that have NaNs in them. Monthly income is a variable number, so we can fill NaNs in with the mean
# value for that column. 
fill_values(df, "MonthlyIncome", fill_val="mean")

,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,60644,2,0.802982,9120.00,13,0,6,0,2.0
1,2,0,0.957151,40,60637,0,0.121876,2600.00,4,0,0,0,1.0
2,3,0,0.658180,38,60601,1,0.085113,3042.00,2,1,0,0,0.0
3,4,0,0.233810,30,60601,0,0.036050,3300.00,5,0,0,0,0.0
4,5,0,0.907239,49,60625,1,0.024926,63588.00,7,0,1,0,0.0
5,6,0,0.213179,74,60629,0,0.375607,3500.00,3,0,1,0,1.0
6,7,0,0.305682,57,60637,0,5710.000000,6670.22,8,0,3,0,0.0
7,8,0,0.754464,39,60625,0,0.209940,3500.00,8,0,0,0,0.0
8,9,0,0.116951,27,60804,0,46.000000,6670.22,2,0,0,0,NaN
9,10,0,0.189169,57,60629,0,0.606291,23684.00,9,0,4,0,2.0


In [66]:
# Number of dependents is always a whole number (you can't have half of a child). It is reasonable to assume that 
# entries with no filled in numbre of dependents means that the person taking the survey simply didn't have
# dependents. So we can fill in the NaNs with the value 0. 
fill_values(df, "NumberOfDependents", fill_val=0)

,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,45,60644,2,0.802982,9120.00,13,0,6,0,2.0
1,2,0,0.957151,40,60637,0,0.121876,2600.00,4,0,0,0,1.0
2,3,0,0.658180,38,60601,1,0.085113,3042.00,2,1,0,0,0.0
3,4,0,0.233810,30,60601,0,0.036050,3300.00,5,0,0,0,0.0
4,5,0,0.907239,49,60625,1,0.024926,63588.00,7,0,1,0,0.0
5,6,0,0.213179,74,60629,0,0.375607,3500.00,3,0,1,0,1.0
6,7,0,0.305682,57,60637,0,5710.000000,6670.22,8,0,3,0,0.0
7,8,0,0.754464,39,60625,0,0.209940,3500.00,8,0,0,0,0.0
8,9,0,0.116951,27,60804,0,46.000000,6670.22,2,0,0,0,0.0
9,10,0,0.189169,57,60629,0,0.606291,23684.00,9,0,4,0,2.0


In [82]:
# We can discretize the MonthlyIncome column by categorizing income into buckets. 
# Let's check the max and min first:
get_max_and_min(df, "MonthlyIncome")

(0.0, 3008750.0)

In [83]:
#Let's do ten buckets.
discretize(df, "MonthlyIncome", 10)

0.0
3008750.0
300875.0


,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,1,1,0.766127,9,60644,2,0.802982,300875.0,13,0,6,0,2.0
1,2,0,0.957151,9,60637,0,0.121876,300875.0,4,0,0,0,1.0
2,3,0,0.658180,9,60601,1,0.085113,300875.0,2,1,0,0,0.0
3,4,0,0.233810,9,60601,0,0.036050,300875.0,5,0,0,0,0.0
4,5,0,0.907239,9,60625,1,0.024926,300875.0,7,0,1,0,0.0
5,6,0,0.213179,9,60629,0,0.375607,300875.0,3,0,1,0,1.0
6,7,0,0.305682,9,60637,0,5710.000000,300875.0,8,0,3,0,0.0
7,8,0,0.754464,9,60625,0,0.209940,300875.0,8,0,0,0,0.0
8,9,0,0.116951,9,60804,0,46.000000,300875.0,2,0,0,0,0.0
9,10,0,0.189169,9,60629,0,0.606291,300875.0,9,0,4,0,2.0


In [84]:
# We can turn the Dependents column into dummy columns. 
make_dummies(df, ["NumberOfDependents"])

,PersonID,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,zipcode,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,...,NumberOfDependents_3.0,NumberOfDependents_4.0,NumberOfDependents_5.0,NumberOfDependents_6.0,NumberOfDependents_7.0,NumberOfDependents_8.0,NumberOfDependents_9.0,NumberOfDependents_10.0,NumberOfDependents_13.0,NumberOfDependents_20.0
0,1,1,0.766127,9,60644,2,0.802982,300875.0,13,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,0.957151,9,60637,0,0.121876,300875.0,4,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0.658180,9,60601,1,0.085113,300875.0,2,1,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0.233810,9,60601,0,0.036050,300875.0,5,0,...,0,0,0,0,0,0,0,0,0,0
4,5,0,0.907239,9,60625,1,0.024926,300875.0,7,0,...,0,0,0,0,0,0,0,0,0,0
5,6,0,0.213179,9,60629,0,0.375607,300875.0,3,0,...,0,0,0,0,0,0,0,0,0,0
6,7,0,0.305682,9,60637,0,5710.000000,300875.0,8,0,...,0,0,0,0,0,0,0,0,0,0
7,8,0,0.754464,9,60625,0,0.209940,300875.0,8,0,...,0,0,0,0,0,0,0,0,0,0
8,9,0,0.116951,9,60804,0,46.000000,300875.0,2,0,...,0,0,0,0,0,0,0,0,0,0
9,10,0,0.189169,9,60629,0,0.606291,300875.0,9,0,...,0,0,0,0,0,0,0,0,0,0


In [107]:
# Parts 5 and 6: Build and Evaluate Classifier
def log_reg(file, dis_var, num_buckets, bin_var):
    '''
    Uses patsy and sklearn to perform a logistic regression on data.

    Inputs:
            file: csv file to be read
            dis_var: variable (column name) to be discretized
            num_buckets: number of intervals for discrete variable
            bin_var: variable (column name) to be turned into dummy var

    Returns: Accuracy score for the regression
    '''
    df = get_df(file)
    filled_df_1 = fill_values(df, "MonthlyIncome", fill_val="mean")
    filled_df_2 = fill_values(filled_df_1, "NumberOfDependents", fill_val=0)
    dis_df = discretize(filled_df_2, dis_var, num_buckets)
    bin_df = make_dummies(dis_df, [bin_var])
    # I learned how to do the following from this example:
    # http://nbviewer.jupyter.org/gist/justmarkham/6d5c061ca5aee67c4316471f8c2ae976
    # Not sure if this is what we were supposed to do or not
    y, X = dmatrices('SeriousDlqin2yrs ~ RevolvingUtilizationOfUnsecuredLines + age + '
                     'DebtRatio + NumberOfOpenCreditLinesAndLoans + zipcode + '
                     'MonthlyIncome + NumberOfOpenCreditLinesAndLoans + NumberOfTimes90DaysLate',
                     bin_df, return_type="dataframe")
    y = np.ravel(y)
    model = LogisticRegression()
    model_fitted = model.fit(X, y)
    return model_fitted.score(X, y)


In [108]:
log_reg("credit-data.csv", "MonthlyIncome", 10, "NumberOfDependents")

0.93315999999999999